In [9]:
# =======================================
# FINAL COMPLETE ASSIGNMENT SOLUTION CODE
# =======================================

# Step 0: Install required libraries
!pip install --quiet gdown seaborn scikit-learn scipy matplotlib pandas

# Step 1: Setup folders
import os
from pathlib import Path

candidate_name = "Syed_Sadain"
root = Path(f"/content/ds_{candidate_name}")
folders = [root, root / "csv_files", root / "outputs"]
for f in folders:
    f.mkdir(parents=True, exist_ok=True)
print("Root folder created at:", root)

# Step 2: Download datasets from Google Drive
import gdown

csv_dir = root / "csv_files"
trader_id = "1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs"
sentiment_id = "1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf"

trader_out = csv_dir / "historical_trader_raw"
sentiment_out = csv_dir / "fear_greed_raw.csv"

gdown.download(f"https://drive.google.com/uc?id={trader_id}", str(trader_out), quiet=False)
gdown.download(f"https://drive.google.com/uc?id={sentiment_id}", str(sentiment_out), quiet=False)

# Step 3: Unzip if needed
import zipfile

files_in_dir = list(csv_dir.iterdir())
zips = [f for f in files_in_dir if f.suffix.lower() == ".zip"]
if zips:
    for z in zips:
        with zipfile.ZipFile(z, 'r') as zip_ref:
            zip_ref.extractall(csv_dir)
    print("Extracted:", zips)

# Step 4: Load CSVs
import pandas as pd

csvs = list(csv_dir.glob("*.csv"))
df_sent = pd.read_csv(sentiment_out)

df_trader = None
trader_path = None
for f in csvs:
    if f != sentiment_out:
        if df_trader is None:
            df_trader = pd.read_csv(f, low_memory=False)
            trader_path = f
        else:
            if f.stat().st_size > trader_path.stat().st_size:
                df_trader = pd.read_csv(f, low_memory=False)
                trader_path = f

if trader_path is None:
    trader_path = trader_out
    df_trader = pd.read_csv(trader_out, low_memory=False)

print("Trader file:", trader_path)
print("Sentiment file:", sentiment_out)

# Step 5: Clean column names
def clean_cols(df):
    df = df.copy()
    df.columns = (df.columns
                  .str.strip()
                  .str.replace(" ", "_")
                  .str.replace("-", "_")
                  .str.replace(".", "")
                  .str.lower())
    return df

df_trader = clean_cols(df_trader)
df_sent = clean_cols(df_sent)

# Step 6: Parse dates
time_cols = [c for c in df_trader.columns if "time" in c or "date" in c or "timestamp" in c]
df_trader[time_cols[0]] = pd.to_datetime(df_trader[time_cols[0]], errors='coerce', utc=True)
df_trader['trade_date'] = df_trader[time_cols[0]].dt.date

date_cols = [c for c in df_sent.columns if "date" in c]
df_sent[date_cols[0]] = pd.to_datetime(df_sent[date_cols[0]], errors='coerce').dt.date
df_sent = df_sent.rename(columns={date_cols[0]: "date"})

if "classification" not in df_sent.columns:
    for c in df_sent.columns:
        if df_sent[c].nunique() < 10:
            df_sent = df_sent.rename(columns={c: "classification"})
            break

# Step 7: Daily aggregates with flexible column detection
pnl_col = next((c for c in df_trader.columns if "pnl" in c or "profit" in c), None)
size_col = next((c for c in df_trader.columns if "size" in c or "qty" in c), None)
leverage_col = next((c for c in df_trader.columns if "leverage" in c), None)
account_col = next((c for c in df_trader.columns if "account" in c), None)
symbol_col = next((c for c in df_trader.columns if any(x in c for x in ["symbol","pair","instrument","ticker"])), None)

for col in [pnl_col, size_col, leverage_col]:
    if col:
        df_trader[col] = pd.to_numeric(df_trader[col], errors='coerce')

group = df_trader.groupby(df_trader['trade_date'])
daily = pd.DataFrame()
daily['num_trades'] = group.size()

if size_col:
    daily['volume'] = group[size_col].sum()
if pnl_col:
    daily['pnl_sum'] = group[pnl_col].sum()
    daily['pnl_mean'] = group[pnl_col].mean()
    daily['pnl_median'] = group[pnl_col].median()
if leverage_col:
    daily['leverage_mean'] = group[leverage_col].mean()
if account_col:
    daily['unique_accounts'] = group[account_col].nunique()
if symbol_col:
    daily['unique_symbols'] = group[symbol_col].nunique()
if pnl_col:
    daily['win_rate'] = group.apply(lambda x: (x[pnl_col] > 0).sum(), include_groups=False) / daily['num_trades']

daily = daily.reset_index().rename(columns={'trade_date': 'date'})
daily.to_csv(csv_dir / "daily_aggregates.csv", index=False)

# Step 8: Merge with sentiment
daily['date'] = pd.to_datetime(daily['date']).dt.date
df_sent['date'] = pd.to_datetime(df_sent['date']).dt.date
merged = pd.merge(daily, df_sent[['date', 'classification']], on='date', how='left')
merged['classification'] = merged['classification'].ffill().bfill()
merged.to_csv(csv_dir / "daily_with_sentiment.csv", index=False)

# Step 9: Visualization (outputs folder guaranteed)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

outputs_dir = root / "outputs"
outputs_dir.mkdir(parents=True, exist_ok=True)

plt.figure(figsize=(10,5))
sns.boxplot(x='classification', y='pnl_sum', data=merged)
plt.title("Daily PnL Sum by Sentiment")
plt.savefig(outputs_dir / "pnl_by_sentiment.png")
plt.close()

plt.figure(figsize=(10,5))
sns.barplot(x='classification', y='win_rate', data=merged)
plt.title("Win Rate by Sentiment")
plt.savefig(outputs_dir / "winrate_by_sentiment.png")
plt.close()

# Step 10: Statistical tests
from scipy import stats
greed = merged[merged['classification'].str.lower().str.contains("greed")]
fear = merged[merged['classification'].str.lower().str.contains("fear")]

u_stat, p_val = stats.mannwhitneyu(
    greed['pnl_sum'].dropna(),
    fear['pnl_sum'].dropna(),
    alternative='two-sided'
)
print(f"Mann-Whitney U test (PnL Sum): U={u_stat}, p={p_val}")

# Step 11: Simple Model - only using available features
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

merged['profitable_day'] = (merged['pnl_sum'] > 0).astype(int)
desired_features = ['num_trades','volume','leverage_mean','win_rate','unique_accounts']
features = [f for f in desired_features if f in merged.columns]

if len(features) == 0:
    raise ValueError("No valid numeric features found for modelling.")

df_model = merged.dropna(subset=features + ['profitable_day'])
X = df_model[features]
y = df_model['profitable_day']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train_sc, y_train)
y_pred = clf.predict(X_test_sc)
y_prob = clf.predict_proba(X_test_sc)[:, 1]

print("Classification Report:\n", classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_prob))
print("\nAll outputs saved in:", outputs_dir)




Root folder created at: /content/ds_Syed_Sadain


Downloading...
From: https://drive.google.com/uc?id=1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs
To: /content/ds_Syed_Sadain/csv_files/historical_trader_raw
100%|██████████| 47.5M/47.5M [00:00<00:00, 201MB/s]
Downloading...
From: https://drive.google.com/uc?id=1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf
To: /content/ds_Syed_Sadain/csv_files/fear_greed_raw.csv
100%|██████████| 90.8k/90.8k [00:00<00:00, 62.8MB/s]


Trader file: /content/ds_Syed_Sadain/csv_files/daily_with_sentiment.csv
Sentiment file: /content/ds_Syed_Sadain/csv_files/fear_greed_raw.csv
Mann-Whitney U test (PnL Sum): U=2949.0, p=0.5355361942691219
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        34

    accuracy                           1.00        47
   macro avg       1.00      1.00      1.00        47
weighted avg       1.00      1.00      1.00        47

ROC AUC: 1.0

All outputs saved in: /content/ds_Syed_Sadain/outputs
